In [23]:
import tensorflow as tf

In [24]:
import numpy as np

In [25]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [26]:
def preprocess_image(image):
    image = tf.image.resize(image,(120,120))
    image=tf.image.adjust_brightness(image,0.2)
    image=tf.image.random_contrast(image, lower=0.5, upper=2.0)
    return image

In [27]:
datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,preprocessing_function=preprocess_image,validation_split=0.2)

In [28]:
train_data=datagen.flow_from_directory("D:\\IMAGE DISEASES\\archive (25)\\Augumented Data",batch_size=32,class_mode="categorical",target_size=(120,120),subset="training")

Found 36617 images belonging to 28 classes.


In [29]:
test_data=datagen.flow_from_directory("D:\\IMAGE DISEASES\\archive (25)\\Augumented Data",batch_size=32,class_mode="categorical",target_size=(120,120),subset="validation")

Found 9142 images belonging to 28 classes.


In [30]:
cnn=tf.keras.models.Sequential()

In [31]:
cnn.add(tf.keras.layers.Conv2D(activation="relu",kernel_size=3,filters=64,input_shape=[120,120,3]))

C:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [32]:
cnn.add(tf.keras.layers.MaxPool2D())

In [33]:
cnn.add(tf.keras.layers.Conv2D(activation="relu",kernel_size=3,filters=64))

In [34]:
cnn.add(tf.keras.layers.MaxPool2D())

In [35]:
cnn.add(tf.keras.layers.Dropout(0.5))

In [36]:
cnn.add(tf.keras.layers.Flatten())

In [37]:
cnn.add(tf.keras.layers.Dense(activation='relu',units=128))

In [38]:
cnn.add(tf.keras.layers.Dense(units=28,activation='softmax'))

In [39]:
cnn.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])

In [40]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

In [41]:
cnn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 118, 118, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 59, 59, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 57, 57, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 28, 28, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 28, 28, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 50176)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       6,422,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 28)                  │           3,612 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,464,988 (24.66 MB)

 Trainable params: 6,464,988 (24.66 MB)

 Non-trainable params: 0 (0.00 B)

In [42]:
cnn.fit(train_data,validation_data=test_data,epochs=60,callbacks=[early_stopping, lr_reduction])

Epoch 1/60


C:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1145/1145 ━━━━━━━━━━━━━━━━━━━━ 401s 349ms/step - accuracy: 0.3701 - loss: 1.7667 - val_accuracy: 0.5393 - val_loss: 1.1250 - learning_rate: 0.0010
Epoch 2/60
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 400s 349ms/step - accuracy: 0.5515 - loss: 1.0971 - val_accuracy: 0.5730 - val_loss: 1.0398 - learning_rate: 0.0010
Epoch 3/60
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 436s 380ms/step - accuracy: 0.5950 - loss: 1.0029 - val_accuracy: 0.6075 - val_loss: 0.9511 - learning_rate: 0.0010
Epoch 4/60
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 435s 379ms/step - accuracy: 0.6187 - loss: 0.9328 - val_accuracy: 0.6267 - val_loss: 0.9256 - learning_rate: 0.0010
Epoch 5/60
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 397s 346ms/step - accuracy: 0.6400 - loss: 0.8779 - val_accuracy: 0.6291 - val_loss: 0.9290 - learning_rate: 0.0010
Epoch 6/60
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 397s 346ms/step - accuracy: 0.6476 - loss: 0.8514 - val_accuracy: 0.6326 - val_loss: 0.8874 - learning_rate: 0.0010
Epoch 7/60
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 401s 349ms/step - acc

In [43]:
cnn.save("new_model.h5")

In [ ]:
# from tensorflow.keras.models import load_model
# from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# model = load_model('model.h5')
# model.add(Dense(256, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)))  
# model.add(Dropout(0.5))  
# model.add(Dense(28, activation='softmax'))  

# model.compile(optimizer=tf.keras.optimizers.Adam(),
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
# history = model.fit(
#     x=train_data,
#     epochs=15,
#     validation_data=test_data,
#     callbacks=[early_stopping, lr_reduction]
# ).
# model.save('modified_model.h5')

In [ ]:
# from tensorflow.keras.models import load_model
# from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# import tensorflow as tf

# model = load_model('modified2_model.h5')

# model.add(Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001), name="dense_custom_2"))
# model.add(Dropout(0.5, name="dropout_custom_2"))
# model.add(Dense(28, activation='softmax', name="output_custom_1"))

# learning_rate = 1e-4
# optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# model.compile(optimizer=optimizer,
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# # early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# # lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# history = model.fit(
#     x=train_data,
#     epochs=15,
#     validation_data=test_data,
#     # callbacks=[early_stopping, lr_reduction]
# )

# model.save('modified3_model.h5')


In [47]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
model = load_model('new_model.h5')
predictions = model.predict(test_data)
predicted_classes = np.argmax(predictions, axis=1)

286/286 ━━━━━━━━━━━━━━━━━━━━ 101s 353ms/step


In [48]:
label_map = {v: k for k, v in train_data.class_indices.items()}
predicted_labels = [label_map[k] for k in predicted_classes]

In [49]:
# predicted_labels.count('RF - AMD')

In [50]:
unique_list = []
for item in predicted_labels:
    if item not in unique_list:
        unique_list.append(item)


In [51]:
unique_list.sort()

In [52]:
unique_list

['Brain - Alzheimer MildDemented',
 'Brain - Alzheimer ModerateDemented',
 'Brain - Alzheimer NonDemented',
 'Brain - Alzheimer VeryMildDemented',
 'Brain - No  Brain Tumor',
 'Brain - glioma Brain Tumor',
 'Brain - meningioma Brain Tumor',
 'Brain - pituitary Brain Tumor',
 'Chest - Covid19',
 'Chest - Lung Opacity',
 'Chest - Normal',
 'Chest - Pneumonia',
 'RF - AMD',
 'RF - Cataract',
 'RF - Glaucoma',
 'RF - Hypertensive Retinopathy',
 'RF - Mild DR',
 'RF - Moderate DR',
 'RF - Normal Fundus',
 'RF - Proliferate DR',
 'RF - Severe DR',
 'Skin - akiec',
 'Skin - bcc',
 'Skin - bkl',
 'Skin - df',
 'Skin - mel',
 'Skin - nv',
 'Skin - vasc']

In [53]:
label_map = train_data.class_indices
print(label_map)

{'Brain - Alzheimer MildDemented': 0, 'Brain - Alzheimer ModerateDemented': 1, 'Brain - Alzheimer NonDemented': 2, 'Brain - Alzheimer VeryMildDemented': 3, 'Brain - No  Brain Tumor': 4, 'Brain - glioma Brain Tumor': 5, 'Brain - meningioma Brain Tumor': 6, 'Brain - pituitary Brain Tumor': 7, 'Chest - Covid19': 8, 'Chest - Lung Opacity': 9, 'Chest - Normal': 10, 'Chest - Pneumonia': 11, 'RF - AMD': 12, 'RF - Cataract': 13, 'RF - Glaucoma': 14, 'RF - Hypertensive Retinopathy': 15, 'RF - Mild DR': 16, 'RF - Moderate DR': 17, 'RF - Normal Fundus': 18, 'RF - Proliferate DR': 19, 'RF - Severe DR': 20, 'Skin - akiec': 21, 'Skin - bcc': 22, 'Skin - bkl': 23, 'Skin - df': 24, 'Skin - mel': 25, 'Skin - nv': 26, 'Skin - vasc': 27}


In [55]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
img = load_img('D:\\IMAGE DISEASES\\archive (25)\\Augumented Data\\Brain - glioma Brain Tumor\\glioma Brain Tumor_0_36.jpeg', target_size=(120, 120))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)[0]

inverse_label_map = {v: k for k, v in label_map.items()}
predicted_label = inverse_label_map[predicted_class]

print(f'Predicted label: {predicted_label}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Predicted label: Brain - glioma Brain Tumor
